<a href="https://colab.research.google.com/github/Salimbhamida/IRM-IA/blob/main/Clustering_lab3_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clustering of a pseudo real dataset using KMeans**

<u>To do :</u>

Before applying Kmeans for the clustering, the dataset requires a <font color='red'>2-steps processing</font> :
- <font color='red'>Processing 1 :</font> Transformation of the <font color='red'>raw dataset</font> into a <font color='red'>numerical format</font>
- <font color='red'>Processing 2 :</font> Improve the quality of the numerical dataset and get a better version

Once processing is completed, we apply the <font color='red'>clustering</font> on the processed dataset.

<u>Note :</u> Replace whenever you find #? by the required python code

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**1. Understand & load the dataset**

The dataset required for a machine learning problem is often a data table where :
- The lines represent members of the target population.
- The columns represent the characteristics/properties of the members.

It is typically stored in a CSV file.

In Python, the <font color='red'>Pandas library</font> allows you to load the data table as a dataframe.

A dataframe is a two-dimensional data structure  whcih  is made up of 3 parts:
- index: line indices
- columns: column indices
- values: data matrix

In dataframe, the columns often have different types in their raw state: integers, real numbers, character strings, dates, etc.

In [41]:
# Import pandas module and give it as alias pd
import pandas as pd

In [42]:
# Load the dataset from CSV file into a pandas dataframe
# The first posotional argument is the data path
# header=0 means that the columns names of dataframe are taken from the first row in CSV file
# index_col=0 means that the index of dataframe rows are taken from the first column of CSV file

df_employes = pd.read_csv('/content/drive/MyDrive/employes.csv', header=0, index_col=0)

In [43]:
# Display the 3 first rows of dataframe
df_employes.head(3)

,salaire,prime,etat civil,date recrutement
nom,,,,
Ali,1200.675,100.560,celibataire,20/10/2018
Sonia,2800.786,400.876,marie,30/11/2005
Rahma,NaN,130.987,celibataire,04/02/2017


In [44]:
# dataframe index
df_employes.index

Index(['Ali', 'Sonia', 'Rahma', 'Salma', 'Ahmed', 'Saleh', 'Ameni', 'Mehdi',
       'Salem', 'Sameh', 'Wajdi', 'Moufida'],
      dtype='object', name='nom')

In [45]:
# dataframe columns
df_employes.columns

Index(['salaire', 'prime', 'etat civil', 'date recrutement'], dtype='object')

In [46]:
# dataframe data values
df_employes.values

array([[1200.675, 100.56, 'celibataire', '20/10/2018'],
       [2800.786, 400.876, 'marie', '30/11/2005'],
       [nan, 130.987, 'celibataire', '04/02/2017'],
       [2500.876, 340.6, 'marie', '03/02/2010'],
       [3100.76, nan, 'marie', '09/09/2004'],
       [1300.876, 150.9, 'celibataire', '01/01/2017'],
       [1100.66, 130.87, 'celibataire', '11/05/2019'],
       [3000.76, nan, 'marie', '07/07/2000'],
       [1505.76, 159.7, 'celibataire', '21/11/2017'],
       [2700.33, 400.0, 'marie', '19/02/1999'],
       [1200.98, nan, 'celibataire', '27/11/2015'],
       [3700.44, 500.0, 'marie', '09/12/1991']], dtype=object)

In [47]:
# Using iloc operator, we can select any part of dataframe based on numerical indices
# For example, select the first column
df_employes.iloc[:,0]

nom
Ali        1200.675
Sonia      2800.786
Rahma           NaN
Salma      2500.876
Ahmed      3100.760
Saleh      1300.876
Ameni      1100.660
Mehdi      3000.760
Salem      1505.760
Sameh      2700.330
Wajdi      1200.980
Moufida    3700.440
Name: salaire, dtype: float64

In [48]:
# Using loc operator,we can select any part of dataframe based on row and column labels
# For example, select the 'salaire' column
df_employes.loc[:,'salaire']

nom
Ali        1200.675
Sonia      2800.786
Rahma           NaN
Salma      2500.876
Ahmed      3100.760
Saleh      1300.876
Ameni      1100.660
Mehdi      3000.760
Salem      1505.760
Sameh      2700.330
Wajdi      1200.980
Moufida    3700.440
Name: salaire, dtype: float64

In [49]:
# For example, we can select 'prime' and 'etat civil' of 'Salma' and 'Ahmed'
df_employes.loc[['Salma','Ahmed'],['prime','etat civil']]

,prime,etat civil
nom,,
Salma,340.6,marie
Ahmed,NaN,marie


In [50]:
# To get 'salaire' column
df_employes.loc[:,'salaire']

nom
Ali        1200.675
Sonia      2800.786
Rahma           NaN
Salma      2500.876
Ahmed      3100.760
Saleh      1300.876
Ameni      1100.660
Mehdi      3000.760
Salem      1505.760
Sameh      2700.330
Wajdi      1200.980
Moufida    3700.440
Name: salaire, dtype: float64

In [51]:
# To get 'salaire' column
df_employes['salaire']

nom
Ali        1200.675
Sonia      2800.786
Rahma           NaN
Salma      2500.876
Ahmed      3100.760
Saleh      1300.876
Ameni      1100.660
Mehdi      3000.760
Salem      1505.760
Sameh      2700.330
Wajdi      1200.980
Moufida    3700.440
Name: salaire, dtype: float64

In [52]:
# Create a copy of the dataframe denoted df_
# The subsequent processing will be performed on the copy
df_ = df_employes.copy()

**2. Processing 1 : Transformation of the raw dataset into a numerical format**

To do it, we follow these steps :
1. We replace the <font color='red'>NaN</font> values by appropriate values
2. The <font color='red'>datetime column</font> is replaced by a column of integer values (like a 'annee recrutement' or 'anciennete' or ...)
3. The <font color='red'>discerete values</font> of column 'etat civil' are replaced by numerical values encoded using <font color='red'>One Hot Encoding (OHE)</font>

**1.1. Replace Missing values (NaN)**

There are often some missing values in the dataset.

When the dataset is loaded, these missing values are represented as NaN values by pandas.



In [53]:
# Compute mean values of 'salaire' and 'prime' columns
mean_salaire=df_['salaire'].mean()
mean_prime=df_['prime'].mean()

In [54]:
mean_salaire

2192.0820909090908

In [55]:
mean_prime

257.1658888888889

In [60]:
# Replace NaN values in 'salaire' and 'prime' colulmns by mean values

# We call df_.fillna() function :
# Its arguments are :
# - a dictionary that contains pairs as many as columns that have NaN values:
#  => Each pair is composed of name_column (as key) and mean_value (as value)
# - inplace=True : means that the replacement is performed on df_employes itself
values = {"salaire": mean_salaire, "prime": mean_prime}
df_.fillna(value=values,
                    inplace=True)

In [57]:
# Check that NaN values are replaced in the dataframe
df_

,salaire,prime,etat civil,date recrutement
nom,,,,
Ali,1200.675000,100.560000,celibataire,20/10/2018
Sonia,2800.786000,400.876000,marie,30/11/2005
Rahma,2192.082091,130.987000,celibataire,04/02/2017
Salma,2500.876000,340.600000,marie,03/02/2010
Ahmed,3100.760000,257.165889,marie,09/09/2004
Saleh,1300.876000,150.900000,celibataire,01/01/2017
Ameni,1100.660000,130.870000,celibataire,11/05/2019
Mehdi,3000.760000,257.165889,marie,07/07/2000
Salem,1505.760000,159.700000,celibataire,21/11/2017


**1.2. Replace the datetime column 'date recrutement' by a numerical column**

In [58]:
# Check the data type of 'date recrutement' column
df_.dtypes

salaire             float64
prime               float64
etat civil           object
date recrutement     object
dtype: object

In [62]:
# Convert the data type of the 'date recrutement' column to 'datetime' type
# Use the function pd.to_datetime() and pass the column 'date recrutement' as argument

df_['date recrutement']=pd.to_datetime(df_['date recrutement'])

<ipython-input-62-b06b2349cc68>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_['date recrutement']=pd.to_datetime(df_['date recrutement'])


In [63]:
# Check the data type of 'date recrutement' column after convertion
df_.dtypes

salaire                    float64
prime                      float64
etat civil                  object
date recrutement    datetime64[ns]
dtype: object

In [70]:
# Create a new column 'anciennete' computed from 'date recrutement' column
# anciennete = annee courante - annee recrutement
# To compute a new column from an existing column, use this trick :
# new_column = exist_column.apply(lambda_expression)
# The lambda expression is applied on each value of exist_column to generate a value in the new column
# Example :define a lambda expression called square that computes and returns the square value of x is defined as follows :
#          square = lambda x : x**2

# Import datetime data type from datetime module
from datetime import datetime

# The following tricks would help you :
# - datetime.now() : returns the current datetime
# - let d be a datetime variable, then d.year is the year (int)

df_['anciennete'] = df_['date recrutement'].apply(lambda x:datetime.now().year-x.year)
df_

,salaire,prime,etat civil,date recrutement,anciennete
nom,,,,,
Ali,1200.675000,100.560000,celibataire,2018-10-20,5
Sonia,2800.786000,400.876000,marie,2005-11-30,18
Rahma,2192.082091,130.987000,celibataire,2017-04-02,6
Salma,2500.876000,340.600000,marie,2010-03-02,13
Ahmed,3100.760000,257.165889,marie,2004-09-09,19
Saleh,1300.876000,150.900000,celibataire,2017-01-01,6
Ameni,1100.660000,130.870000,celibataire,2019-11-05,4
Mehdi,3000.760000,257.165889,marie,2000-07-07,23
Salem,1505.760000,159.700000,celibataire,2017-11-21,6


In [72]:
# The column 'date recrutement' is no longer useful
# Logically, we drop it
# Call drop() function and pass the following arguments :
# - the column name to be dropped
# - the axis either 0 (if we drop rows) or or 1 (if we drop columns)
df_.drop('date recrutement', axis=1, errors='ignore', inplace=True)
df_

,salaire,prime,etat civil,anciennete
nom,,,,
Ali,1200.675000,100.560000,celibataire,5
Sonia,2800.786000,400.876000,marie,18
Rahma,2192.082091,130.987000,celibataire,6
Salma,2500.876000,340.600000,marie,13
Ahmed,3100.760000,257.165889,marie,19
Saleh,1300.876000,150.900000,celibataire,6
Ameni,1100.660000,130.870000,celibataire,4
Mehdi,3000.760000,257.165889,marie,23
Salem,1505.760000,159.700000,celibataire,6


**1.3. Convertion of the discrete values of 'etat civil' column**

In [73]:
# Use the OneHotEncoder (OHE) to encode 'etat civil' column
# Each value is encoded to a binary vector
# 'celibataire' => [1,0]
# 'marie'       => [0,1]

# Import the OneHotEncoder class from sklearn.preprocessing module
from sklearn.preprocessing import OneHotEncoder

In [74]:
# Create an instance ohe from OneHotEncoder
# No hyperparameters are required
ohe=OneHotEncoder()

In [78]:
# Call fit() function and pass 'etat civil' column as an argument
# The 'etat civil' column should be specified as a dataframe
ohe.fit(df_[['etat civil']])

OneHotEncoder()

In [79]:
# Show the OHE model
# The model parameters are the unique values to be encoded, called categories
categories = ohe.categories_[0]
categories

array(['celibataire', 'marie'], dtype=object)

In [81]:
# Transform the discete values of 'etat civil' column
# Call transform() function and pass 'etat civil' column (as a dataframe) as an argument
# Normally, the encoded 'etat civil' would be a binary matrix
# However, OHE returns it as a sparse matrix (to optimize the memory storage)
encoded_etat_civil=ohe.transform(df_[['etat civil']])
encoded_etat_civil

<12x2 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [82]:
# Convert the sparse matrix to a numpy matrix
# Use toarray()function
encoded_etat_civil=encoded_etat_civil.toarray()
encoded_etat_civil

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [92]:
# Convert the numpy matrix of encoded 'etat civil' to a dataframe
# Use pandas DataFrame class
# Pass the following arguments :
# - numpy array of encoded etat civil (already computed)
# - index : the same index as df_
# - columns : concatenate 'etat_civil_' and categories
df_encoded_etat_civil=pd.DataFrame(encoded_etat_civil,
                                    index=['Ali', 'Sonia', 'Rahma', 'Salma', 'Ahmed', 'Saleh', 'Ameni', 'Mehdi',
       'Salem', 'Sameh', 'Wajdi', 'Moufida'],columns = ['etat_civil_' + i for i in categories])
df_encoded_etat_civil

,etat_civil_celibataire,etat_civil_marie
Ali,1.0,0.0
Sonia,0.0,1.0
Rahma,1.0,0.0
Salma,0.0,1.0
Ahmed,0.0,1.0
Saleh,1.0,0.0
Ameni,1.0,0.0
Mehdi,0.0,1.0
Salem,1.0,0.0
Sameh,0.0,1.0


In [95]:
# Concat the encoded 'etat civil' to a dataframe to df_
# Use pandas concat function
# Pass as arguments :
# - a list that contains the old dataframe df_ and the dataframe of encoded etat civil
# - axis : specify the value
#    0 (if the concat is performed in vertical axis)
#    1 (if the concat is performed in horizontal axis)
df_ = pd.concat([df_,df_encoded_etat_civil] ,  axis=0)
df_

,salaire,prime,etat civil,anciennete,etat_civil_celibataire,etat_civil_marie
Ali,1200.675000,100.560000,celibataire,5.0,NaN,NaN
Sonia,2800.786000,400.876000,marie,18.0,NaN,NaN
Rahma,2192.082091,130.987000,celibataire,6.0,NaN,NaN
Salma,2500.876000,340.600000,marie,13.0,NaN,NaN
Ahmed,3100.760000,257.165889,marie,19.0,NaN,NaN
Saleh,1300.876000,150.900000,celibataire,6.0,NaN,NaN
Ameni,1100.660000,130.870000,celibataire,4.0,NaN,NaN
Mehdi,3000.760000,257.165889,marie,23.0,NaN,NaN
Salem,1505.760000,159.700000,celibataire,6.0,NaN,NaN
Sameh,2700.330000,400.000000,marie,24.0,NaN,NaN


In [103]:
# The 'etat civil' column is no longer useful
# We drop it
df_.drop('etat civil',axis=1,inplace=True)
df_

,salaire,prime,anciennete,etat_civil_celibataire,etat_civil_marie
Ali,1200.675000,100.560000,5.0,NaN,NaN
Sonia,2800.786000,400.876000,18.0,NaN,NaN
Rahma,2192.082091,130.987000,6.0,NaN,NaN
Salma,2500.876000,340.600000,13.0,NaN,NaN
Ahmed,3100.760000,257.165889,19.0,NaN,NaN
Saleh,1300.876000,150.900000,6.0,NaN,NaN
Ameni,1100.660000,130.870000,4.0,NaN,NaN
Mehdi,3000.760000,257.165889,23.0,NaN,NaN
Salem,1505.760000,159.700000,6.0,NaN,NaN
Sameh,2700.330000,400.000000,24.0,NaN,NaN


In [104]:
# Save the processed numerical dataset into a CSV file
# Call to_csv() function and pass 'employes_num.csv' as file name
df_.to_csv('employes_num.csv')